# Booking.com Feature Extraction
---

Download the file BookingDotCom_HotelReviews.xlsx from Canvas. This file contains over 515,000 guest reviews and rating of almost 1500 hotels across Europe scraped from popular hotel reservation website Booking.com. The text data was cleaned by removing unicode and punctuation and transformed to lower case. No other preprocessing was done. More information on each field is provided in the "Data Description" tab of the Excel file.

        1. What are the top five hotel features (e.g., location, staff, etc.) that customers mention the most in positive reviews and top five features they mention most in negative reviews? Your identified features must make sense (e.g., "great" or "negative" are not features). (3 points)
        
        2. What are the top five features that customers prefer most if they are a solo traveler vs traveling with a group vs on a business trip vs a leisure trip vs traveling as a couple vs a family with young children. You will find these categories in the "Tags" column. There are a few more tags that we don't need. (2 points).

        3. What are the top five features customers like most and top five features they complain about most about hotels in United Kingdom, France, Italy, and Spain? Country information is available inside Hotel_Address. (2 points)
        
        4. Create a dashboard with the following plots; (1) "Top Five Hotels Overall" with consistently high ratings, (2) Bottom Five Hotels Overall" with consistently low ratings, (3) Five Most Improved Hotels" with the highest improvement in average ratings from 2015 to 2017, showing their average ratings for each of the three years. (0.5+0.5+2 points).

Write clear, compact, and understandable code with comment/markdown statements as appropriate. Non-working code or unnecessary code will be penalized. 

Submit your Jupyter file using the link below or provide a link to your Google Colab or Github file.


In [1]:
# import packages to use
import pandas as pd
import pycountry
import spacy
from spacy import displacy
# from collections import Counter
import en_core_web_sm

In [2]:
# load dataframe
df = pd.read_excel("BookingDotCom_HotelReviews.xlsx", sheet_name="Data")

# sample the first 100 rows of df
df = df[:100]

# rename df columns to lower case
df.columns= df.columns.str.lower()

In [3]:
# convert to lowercase & remove double/unneccessary spaces between words in columns of comments
df['positive_comments'] = df['positive_comments'].str.lower().apply(lambda x: " ".join(x.split()))
df['negative_comments'] = df['negative_comments'].str.lower().apply(lambda x: " ".join(x.split()))

In [5]:
'''
Create column with country name; we get the list of countries from the pycountry package
and use the hotel_address column to extract country name
'''

df['country'] = df["hotel_address"].apply(
    lambda address: ' '.join([c.name for c in pycountry.countries if c.name in address])
    )

# use the review_date column to extract the year and store in new column
df['year'] = pd.DatetimeIndex(df['review_date']).year

In [6]:
'''
In this step we deal with the tags column using the steps defined below:
    1. Define tags we are interested in
    2. Define a function to apply to tags column to remove tags we are not interested in by:
            - Converting the individual row values to list (from string) e.g. "[' Leisure trip ']" -> [' leisure trip ']
            - Strip the whitespaces from individual elements e.g. [' leisure trip ']-> ['leisure trip']
            - Drop tags we are not interested in
'''

# customer tags we are interested in
customer_tags = [
    'solo traveler',
    'group',
    'business trip',
    'leisure trip',
    'couple',
    'family with young children'
]

def clean_tag(x):

    # convert value from string to a list
    import ast
    
    myTags = ast.literal_eval(x.lower())

    # strip whitespaces from elements and drop those we are not interested in
    myTags = [customerTag.strip() for customerTag in myTags if customerTag.strip() in customer_tags]

    return myTags


# apply clean_tag() function to the tags column
df['tags'] = df['tags'].apply(lambda x: clean_tag(x))

In [7]:
'''
Function applied to tags column to extract new columns for customer categories
Lambda function will be used as in the steps below
'''
def split_tag(x:list, tagName:str) -> int:
    
    t = [1 if tagName in x else 0][0]
    
    return t

# dictionary of column names (new additional columns) and customer tags (as contained in tags of interest)
tagDict = {
    'solo_traveler' : 'solo traveler',
    'group' : 'group',
    'business_trip' : 'business trip',
    'leisure_trip' : 'leisure trip',
    'couple' : 'couple',
    'family_with_young_children' : 'family with young children'
}

# applying function on tags column to get new separated columns
for key, value in tagDict.items():
    df[key] = df['tags'].apply(lambda x: split_tag(x, value))

In [8]:
# drop columns we do not need for now
colsToDrop = [
    'hotel_address', 
    'review_date', 
    'reviewer_nationality', 
    'tags'
    ]

df.drop(columns=colsToDrop, inplace=True)

1. Text Processing
- Cleaning
- Normalization
- Tokenization
- Removing stopwords
- POS tagging
- Named Entity Recognition
- Lemmatization

2. Feature Extraction
- Bag of Words
- TF-IDF
- Word embeddings

3. Modeling

# NER

In [31]:
# instantiate Spacy model for performing NER
nlp = en_core_web_sm.load()

# apply model to text
doc = nlp(df.positive_comments[10])
doc.text

'rooms were stunningly decorated and really spacious in the top of the building pictures are of room 300 the true beauty of the building has been kept but modernised brilliantly also the bath was lovely and big and inviting great more for couples restaurant menu was a bit pricey but there were loads of little eatery places nearby within walking distance and the tram stop into the centre was about a 6 minute walk away and only about 3 or 4 stops from the centre of amsterdam would recommend this hotel to anyone it s unbelievably well priced too'

In [34]:
doc.text

'rooms were stunningly decorated and really spacious in the top of the building pictures are of room 300 the true beauty of the building has been kept but modernised brilliantly also the bath was lovely and big and inviting great more for couples restaurant menu was a bit pricey but there were loads of little eatery places nearby within walking distance and the tram stop into the centre was about a 6 minute walk away and only about 3 or 4 stops from the centre of amsterdam would recommend this hotel to anyone it s unbelievably well priced too'

In [30]:
# docs = list(nlp.pipe(df.positive_comments))
# len(docs)

In [18]:
# entities
docEntities = doc.ents
docEntities

(300, 6 minute, only about 3, 4)

In [ ]:
# nounPhrases
nounPhrases = [chunk.text for chunk in doc.noun_chunks]
nounPhrases

In [20]:
pos = [
    'PROPN',
    'NOUN',
    'VERB'
]

In [21]:
# using POS tagging
clean_doc=[token for token in doc if token.pos_ in pos]
print(clean_doc)

[rooms, decorated, top, building, pictures, room, beauty, building, kept, modernised, bath, inviting, couples, restaurant, menu, bit, were, loads, eatery, places, walking, distance, tram, stop, centre, minute, walk, stops, centre, amsterdam, recommend, hotel, s, priced]


In [ ]:
# sentences
spSent = list(doc.sents)

# words
spWords = [w.text for w in doc]

# clean words: remove stopwords and punctuation
spCleanWords = [w.text for w in doc
           if not w.is_stop and not w.is_punct]

In [ ]:
# Word frequency
from collections import Counter
spCleanWordFreq = Counter(spCleanWords)
print(spCleanWordFreq)

In [ ]:
spCleanWordFreq.most_common(5)

In [ ]:
# Words occurring twice or more
spFreqCleanWords = [w for (w, freq) in spCleanWordFreq.items() if freq > 1]
spFreqCleanWords

In [ ]:
# lemmatization
for w in doc:
     print (w, w.lemma_)

In [ ]:
CleanSentence = ' '.join(spCleanWords).strip()
CleanSentence

In [ ]:
spCleanSentence = nlp(CleanSentence)
spCleanWords = []

for w in spCleanSentence:
    spCleanWords.append(w.lemma_)

print(spCleanWords)

In [ ]:
# Extracting nouns using POS tags

spNouns = []
for w in doc:
     if w.pos_ == 'NOUN':
         spNouns.append(w)

spNouns

In [ ]:
# dependency parsing
sentence = doc
spSentence = nlp(sentence)

for w in spSentence:
     print (w.text, w.tag_, w.head.text, w.dep_)

In [ ]:
# shallow parsing
for chunk in doc.noun_chunks:
     print (chunk)

In [ ]:
displacy.render(doc, style='ent', jupyter=True, options={'distance': 75})

In [ ]:
# noun phrase detection
for chunk in doc.noun_chunks:
     print (chunk)

# Coreference resolution
https://towardsdatascience.com/from-text-to-knowledge-the-information-extraction-pipeline-b65e7e30273e

# Bi-grams

https://medium.com/analytics-vidhya/feature-extraction-and-sentiment-analysis-of-reviews-of-3-apps-in-india-84b665e1a887

# Trial

In [ ]:
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

#pip install gensim
from gensim.utils import simple_preprocess, lemmatize
from gensim.corpora import Dictionary

In [ ]:
my_docs = ['This movie was about spaceships and aliens.',
           'I really enjoyed the movie!',
           'Awesome action scenes, but boring characters.',
           'The movie was awful! I hate alien films.',
           'Space is cool! I like space movies.',
           'More space films, please!']

tokens = [] # list of lists >> column of pos/neg reviews tokens
for doc in my_docs:
    words = regexp_tokenize(doc.lower(), r'[A-Za-z]+')
    words = [w for w in words if w not in stopwords.words('english')]
    words = [lemmatizer.lemmatize(w) for w in words]
    tokens.append(words)

my_dict = Dictionary(tokens) # dictionary
print(my_dict.token2id) 

# Note: You can also add new words or lists of words to a dictionary, save a
# dictionary to a file, load it back later, or read a dictionary from a text file

# my_dict.add_documents(list_of_new_words)
# my_dict.save('saved_dict.dict')
# loaded_dict = Dictionary.load('saved_dict.dict')
# dictionary = Dictionary(line.split()) for line in open('sample.txt', encoding='utf-8'))

dtm = [my_dict.doc2bow(doc) for doc in tokens]
dtm  # Create Gensim BOW corpus

for doc in dtm:
    print([[my_dict[i], freq] for i, freq in doc])